In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.metrics import mean_squared_error

from collections import deque

from rashomon import loss
from rashomon import counter
from rashomon.aggregate import RAggregate_profile
from rashomon.sets import RashomonSet, RashomonProblemCache, RashomonSubproblemCache
from rashomon import tva, metrics
from rashomon.extract_pools import extract_pools


%load_ext autoreload
%autoreload 2
# %matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Worst case scenario

In [3]:
sigma = np.array([[1, 1, 0],
                  [0, 1, 0]], dtype='float64')
sigma_profile = (1, 1)

M, n = sigma.shape
R = np.array([5, 5])

num_policies = np.prod(R-1)
all_policies = tva.enumerate_policies(M, R)
policies = [x for x in all_policies if tva.policy_to_profile(x) == sigma_profile]
pi_pools, pi_policies = extract_pools(policies, sigma)

for pool_id, pool in pi_pools.items():
    print(pool_id, ":", [policies[i] for i in pool])
    # print(pool_id, ":", pool)

0 : [(1, 1), (2, 1), (3, 1)]
1 : [(1, 2), (1, 3), (2, 2), (2, 3), (3, 2), (3, 3)]
2 : [(1, 4), (2, 4), (3, 4)]
3 : [(4, 1)]
4 : [(4, 2), (4, 3)]
5 : [(4, 4)]


### Generate data

In [4]:
np.random.seed(3)

num_pools = len(pi_pools)
# # ADVERSARIAL EXAMPLE
mu = np.array([0, 1.5, 3, 3, 6, 4.5])
# CORRELATION?
# mu = np.array([0, 1, 2, 1, 3, 3])
# mu = mu*4
var = [1] * num_pools

n_per_pol = 100
sim_num = 1

num_data = num_policies * n_per_pol
X = np.ndarray(shape=(num_data, M))
D = np.ndarray(shape=(num_data, 1), dtype='int_')
y = np.ndarray(shape=(num_data, 1))

for idx, policy in enumerate(policies):
    pool_i = pi_policies[idx]
    mu_i = mu[pool_i]
    var_i = var[pool_i]
    y_i = np.random.normal(mu_i, var_i, size=(n_per_pol, 1))

    start_idx = idx * n_per_pol
    end_idx = (idx + 1) * n_per_pol

    X[start_idx:end_idx, ] = policy
    D[start_idx:end_idx, ] = idx
    y[start_idx:end_idx, ] = y_i
    

### RAggregate

In [26]:
# Rashomon set estimation

P_set = RAggregate_profile(M, R, 8, D, y, 2, sigma_profile, reg=0.1)
if not P_set.seen(sigma):
    print("P_set missing true sigma")

In [27]:
pol_means = loss.compute_policy_means(D, y, num_policies)
true_best = pi_pools[np.argmax(mu)]
min_dosage_best_policy = metrics.find_min_dosage(true_best, policies)

for s_i in P_set:
    pi_pools_i, pi_policies_i = extract_pools(policies, s_i)
    pool_means_i = loss.compute_pool_means(pol_means, pi_pools_i)
    
    Q = loss.compute_Q(D, y, s_i, policies, pol_means, reg=0.1)
    y_pred = metrics.make_predictions(D, pi_policies_i, pool_means_i)
    sqrd_err = mean_squared_error(y, y_pred)

    # IOU
    pol_max = metrics.find_best_policies(D, y_pred)
    iou = metrics.intersect_over_union(set(true_best), set(pol_max))

    # Min dosage membership
    min_dosage_present = metrics.check_membership(min_dosage_best_policy, pol_max)

    # Best policy difference
    best_pol_diff = np.max(mu) - np.max(pool_means_i)
    
    print(f"Num pools: {len(pi_pools_i)}, Squared loss: {sqrd_err:0.5f}, Q: {Q:0.5f}")
    print(f"Best policy IOU: {iou:.3f}")
    print(f"Min dosage: {min_dosage_present}")
    print(f"Best policy error: {best_pol_diff}")
    print("---")

    this_list = [n_per_pol, sim_num, len(pi_pools_i), sqrd_err, iou, min_dosage_present, best_pol_diff]
    rashomon_list.append(this_list)

Num pools: 4, Squared loss: 1.33086, Q: 1.73086
Best policy IOU: 0.667
Min dosage: True
Best policy error: 0.48808069259229026
---
Num pools: 6, Squared loss: 1.33013, Q: 1.93013
Best policy IOU: 0.667
Min dosage: True
Best policy error: 0.48808069259229026
---
Num pools: 6, Squared loss: 1.33006, Q: 1.93006
Best policy IOU: 0.667
Min dosage: True
Best policy error: 0.48808069259229026
---
Num pools: 6, Squared loss: 1.24759, Q: 1.84759
Best policy IOU: 0.500
Min dosage: True
Best policy error: 0.029118080070194452
---
Num pools: 6, Squared loss: 1.02158, Q: 1.62158
Best policy IOU: 1.000
Min dosage: True
Best policy error: -0.009341137959020251
---
Num pools: 8, Squared loss: 1.02089, Q: 1.82089
Best policy IOU: 0.500
Min dosage: False
Best policy error: -0.04780035598823318
---


In [149]:
col_names = ["n_per_pol", "sim_num", "num_pools", "MSE", "IOU", "min_dosage", "best_pol_MSE"]
rashomon_df = pd.DataFrame(rashomon_list, columns=col_names)

rashomon_df

,n_per_pol,sim_num,num_pools,MSE,IOU,min_dosage,best_pol_MSE
0,100,1,4,1.330859,0.666667,True,0.488081
1,100,1,6,1.330133,0.666667,True,0.488081
2,100,1,6,1.330060,0.666667,True,0.488081
3,100,1,6,1.247588,0.500000,True,0.029118
4,100,1,6,1.021578,1.000000,True,-0.009341
5,100,1,8,1.020892,0.500000,False,-0.047800


### LASSO - Beta -> alpha

In [150]:
G = tva.alpha_matrix(M, R, policies)
# print(G)
D_matrix = tva.get_dummy_matrix(D, G, num_policies)

In [151]:
reg_param = 1e-1
mod1 = linear_model.Lasso(reg_param, fit_intercept=False)
mod1.fit(D_matrix, y)
alpha_est = mod1.coef_

In [152]:
lasso_list = []

In [153]:
y_tva = mod1.predict(D_matrix)

# MSE
sqrd_err = mean_squared_error(y_tva, y)
L1_loss = sqrd_err + reg_param * np.linalg.norm(alpha_est, ord=1)

# IOU
tva_best = metrics.find_best_policies(D, y_tva)
iou_tva = metrics.intersect_over_union(set(true_best), set(tva_best))

# Min dosage inclusion
min_dosage_present_tva = metrics.check_membership(min_dosage_best_policy, tva_best)

# Best policy MSE
best_policy_error_tva = np.max(mu) - np.max(y_tva)

this_list = [n_per_pol, sim_num, sqrd_err, L1_loss, iou_tva, min_dosage_present_tva, best_policy_error_tva]
lasso_list.append(this_list)

In [154]:
lasso_cols = ["n_per_pol", "sim_num", "MSE", "L1_loss", "IOU", "min_dosage", "best_pol_MSE"]
lasso_df = pd.DataFrame(lasso_list, columns=lasso_cols)

lasso_df

,n_per_pol,sim_num,MSE,L1_loss,IOU,min_dosage,best_pol_MSE
0,100,1,1.347481,1.859604,0.0,False,0.878772


In [111]:
print(alpha_est)
np.matmul(G, alpha_est)

[0.00000000e+00 4.02587900e+00 7.29379941e-02 3.72378857e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.36637192e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.83710615e+00 4.10780553e+00 4.10336904e-03 2.56179131e+00]


array([ 0.        ,  4.025879  ,  4.098817  ,  7.82260557,  0.        ,
        4.025879  ,  4.098817  ,  7.90626929,  0.        ,  4.025879  ,
        4.098817  ,  7.90626929,  3.83710615, 11.97079068, 12.04783205,
       18.41707565])

In [164]:
G

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
       [1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0.,

In [112]:
alpha_des = np.copy(alpha_est)
alpha_des[:] = 0

alpha_des[1] = 1
alpha_des[3] = 1
alpha_des[12] = 1
alpha_des[13] = 1
alpha_des[15] = -1
print(alpha_des)

np.matmul(G, alpha_des)

[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0. -1.]


array([0., 1., 1., 2., 0., 1., 1., 2., 0., 1., 1., 2., 1., 3., 3., 3.])

### Causal Trees

In [4]:

from CTL.causal_tree_learn import CausalTree
from sklearn.model_selection import train_test_split

In [5]:
X_constant = np.zeros(shape=D.shape) + 1

In [28]:
y_1d = y.reshape((-1,))
D_1d = D.reshape((-1,))

In [29]:
y.shape

(1600, 1)

In [30]:
x_train, x_test, y_train, y_test, treat_train, treat_test = train_test_split(
    X_constant, y_1d, D_1d, test_size=0.5, random_state=3)

In [33]:
ct_adaptive = CausalTree(weight=0, split_size=0)
ct_adaptive.fit(x_train, y_train, treat_train)
ct_adaptive.prune()
ct_adaptive_predict = ct_adaptive.predict(X_constant)

In [34]:
print(ct_adaptive_predict)

[2.42444038 2.42444038 2.42444038 ... 2.42444038 2.42444038 2.42444038]


In [12]:
asthma = pd.read_csv('CTL-2.43/data/asthma.txt', delimiter=' ', index_col=None)

asthma.columns = ['physician', 'age', 'sex', 'education', 'insurance', 'drug coverage', 'severity',
                  'comorbidity', 'physical comorbidity', 'mental comorbidity', 'satisfaction']

y = asthma['satisfaction'].values
treatment = asthma['physician'].values

x = asthma.drop(['satisfaction', 'physician'], axis=1).values

columns = asthma.drop(['satisfaction', 'physician'], axis=1).columns

y[y == 0] = -1

treatment[treatment == 1] = 0
treatment[treatment == 2] = 1

np.random.seed(0)


x_train, x_test, y_train, y_test, treat_train, treat_test = train_test_split(x, y, treatment,
                                                                             test_size=0.5, random_state=42)

In [14]:
y_test

array([ 1,  1,  1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1,
        1, -1,  1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1, -1,  1, -1, -1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1,
       -1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,
       -1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1,  1,
       -1, -1,  1, -1,  1,  1, -1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1,
        1,  1])